In [ ]:
#Mounting with Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Installing library that converts PDF to Text
pip install PyPDF2

     |████████████████████████████████| 81kB 6.1MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp37-none-any.whl size=61085 sha256=8625a75a9ecc7362169771f6144888fe8145222f8075de60e1f16bb8e0adeb32
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [ ]:
#NLP transformation library
!pip install sentence_transformers

     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 2.3MB 20.2MB/s 
     |████████████████████████████████| 1.2MB 37.9MB/s 
     |████████████████████████████████| 3.3MB 53.1MB/s 
     |████████████████████████████████| 901kB 44.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=3b960da2f6065c186dc0071743252e69f2b9d17ac6f973103c62adea5cbdacef
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [ ]:
#importing necessary libraries 
import PyPDF2
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import random
import numpy as np
from numpy import array
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Function for forcasting the upcoming number of incidents for Incidents 
def prediction(train, test):
  def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
      # find the end of this pattern
      end_ix = i + n_steps
      # check if we are beyond the sequence
      if end_ix > len(sequence)-1:
        break
      # gather input and output parts of the pattern
      seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
      X.append(seq_x)
      y.append(seq_y)
    return array(X), array(y)
  raw_seq = train
# choose a number of time steps
  n_steps = 4
  # split into samples
  X, y = split_sequence(raw_seq, n_steps)
  # reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
  n_features = 1
  n_seq = 2
  n_steps = 2
  X = X.reshape((X.shape[0], n_seq, n_steps, n_features))
  # define model
  model = Sequential()
  model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(Flatten()))
  model.add(LSTM(50, activation='relu'))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mse')
  # fit model
  model.fit(X, y, epochs=500, verbose=0)
  x_input = test
  #x_input = x_input.reshape((1, n_seq, n_steps, n_features))
  ans=[]
  for i in range(0, 4):
    ip=x_input[len(x_input)-4: len(x_input)]
    temp=array(ip)
    ip = temp.reshape((1, n_seq, n_steps, n_features))
    yhat = model.predict(ip, verbose=0)
    x=yhat.tolist()
    ans.append(x[0][0])
    x_input.append(x[0][0])

    #x_input = x_input.reshape((1, n_seq, n_steps, n_features))
  finalOutput={'Prediction:':ans}
  return(finalOutput)

In [ ]:
# Function that returns all the necessary information and risk score factors 
def category(file, loc, sev ,trav, base, riskscore, t):
  #Extraction of Text and converting into string
  if (t=="z"):
    if (file.split(".")[-1]=="pdf"):
      pdfFileObj = open(file, 'rb')
      pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
      text=''
      for i in range(pdfReader.numPages):
          text=text + " " +  str(pdfReader.getPage(i).extractText())
      temptext=text
      text=text.replace("\n","")
    else:
      f = open(file, "r")
      text = f.read()
      temptext=text
  else:
    text=t
    temptext=text
  n_gram_range = (3, 3)
  stop_words = "english"
  #print(text)
  
  #Find relevent keywords from text
  count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([text])
  candidates = count.get_feature_names() #candidate Keywords
  model = SentenceTransformer('distilbert-base-nli-mean-tokens')
  doc_embedding = model.encode([text])
  candidate_embeddings = model.encode(candidates)
  top_n = 50
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]


  incidentDescription=''
  description=[candidates[index] for index in distances.argsort()[0][-10:]]
  for i in description:
    incidentDescription=incidentDescription + " " +i 
  
  #Extracting information Like source,incident dates and subjecy
  source='' #Source
  incidentDate='' #IncidentDate
  subject='' #SubjectLine
  x=re.split('\n', temptext)
  for num,val in enumerate(x):
    if val=='From:':
      source=x[num+1]
    if val =='Sent:':
      incidentDate=x[num+1]
    if 'Subject:' in val:
      subject=val[9:]
  if '@riskadvisory.net' in source:
    source='Risk Advisory'
  elif '@internationalsos.com' in source:
    source='ISOS'
  elif '@everbridge.net':
    source='VCC'
  else:
    source='Core'

  df1=pd.read_excel("/content/drive/MyDrive/EY_Hack_GDS_2.0/Locations and Headcount.xlsx")
  df1.drop_duplicates(subset ="LocationCity",inplace=True)
  cities=list(df1["LocationCity"])
  countries=list(df1.drop_duplicates(subset ="Geo Country")["Geo Country"])
  geoRegion=list(df1.drop_duplicates(subset ="Geo Region")["Geo Region"])
  area=list(df1.drop_duplicates(subset ="Area")["Area"])
  countByCountry=df1.groupby(["Geo Country"]).sum()
  countByCity=df1.groupby(["LocationCity"]).sum()

  affectedCountries=dict()#affected countries
  for country in countries:
    for val in keywords:
      if country.lower() in val or country in val:
        if country in affectedCountries.keys():
          affectedCountries[country]=affectedCountries[country]+1
        else:
          affectedCountries[country]=1
  maxVal=0
  affectedCountry=''
  for i in affectedCountries.keys():
      if affectedCountries[i]>maxVal:
          maxVal=affectedCountries[i]
          affectedCountry=i

  # Predicitng affected City 
  affectedCities=dict() #affected cities
  for city in cities:
    for val in keywords:
      if city.lower() in val or city in val:
        if city in affectedCities.keys():
          affectedCities[city]=affectedCities[city]+1
        else:
          affectedCities[city]=1
  maxVal=0
  affectedCity=''
  for i in affectedCities.keys():
      if affectedCities[i]>maxVal:
          maxVal=affectedCities[i]
          affectedCity=i
  
  # Defining Keywords for the predicting the category of Risk
  categoryMapping={'Core':
  {
      'Political':['political','political stability','war'],
      'Security':['security'],
      'Travel':['travel'],
      'Crime':['crime'],
      'Civil':['civil'],
      'Terrorism':['terrorism'],
      'Medical':['modical'],
      'Infrastructure':['infrastructure'],
      'Kidnap':['Kidnap'],
      'Institutional':['institutional','labour','operational']
  },
            'Risk Advisory':
  {
      'Political':['interstate','regime'],
      'Security':['security','geophysical'],
      'Travel':['road'],
      'Crime':['crime'],
      'Civil':['civil'],
      'Terrorism':['terrorism'],
      'Medical':['medical','covid19'],
      'Infrastructure':['infrastructure'],
      'Kidnap':['Kidnap'],
      'Institutional':['corruption','LGBT','State']
  },
            'VCC':
  {
      'Political':['war'],
      'Security':['earthquake','volcano','flood','winter','disaster','cyclone','hurricane','tornado'],
      'Travel':['accident','flight','marine','transportation'],
      'Crime':['hostage','shooting','robbery','police'],
      'Civil':['civil'],
      'Terrorism':['bomb','suspicious'],
      'Medical':['medical','disease'],
      'Infrastructure':['chemical','wildfire','power','telecommunications','fire','radioactive','avalanche','landslide','subsidence','water','gas','sewege'],
      'Kidnap':['Kidnap'],
      'Institutional':['corruption','LGBT']
  },  
        'ISOS':
            {
      'Political':['political','election','conflict'],
      'Security':['volcano','flood','snowstrom'],
      'Travel':['road','travel','transport','border'],
      'Crime':['crime','police','shooting'],
      'Civil':['civil','strike','protest','rally'],
      'Terrorism':['terrorism','explosion','terror'],
      'Medical':['covid19'],
      'Infrastructure':['infrastructure','wildlife','electricity','internet'],
      'Kidnap':['kidnap'],
      'Institutional':['institutional']
  }
  }
  category={
      'Political':0,
      'Security':0,
      'Travel':0,
      'Crime':0,
      'Civil':0,
      'Terrorism':0,
      'Medical':0,
      'Infrastructure':0,
      'Kidnap':0,
      'Institutional':0
  }
  total=0
  for cat in categoryMapping[source].keys():
      for val in categoryMapping[source][cat]:
          for key in keywords:
              if val in key:
                  total=total+1
                  category[cat]=category[cat]+1
  maxVal=0
  finalCat=''
  for i in category.keys():
      category[i]=int((category[i]/total)*100)
      if category[i]>maxVal:
          maxVal=category[i]
          finalCat=i
  #print(finalCat)
  #Calculating the Severity of the incident
  severity=pd.read_excel("/content/drive/MyDrive/EY_Hack_GDS_2.0/Severity.xlsx")
  severity.set_index('Country ',inplace=True)
  sevValue=severity.loc[[affectedCountry]][finalCat][affectedCountry]
  sevValue
  df1=pd.read_excel("/content/drive/MyDrive/EY_Hack_GDS_2.0/Locations and Headcount.xlsx")
  officeImpacted=df1.groupby('Geo Country').count()
  officeImpacted=officeImpacted.loc[[affectedCountry]]['Area'][affectedCountry]

  numberofEmp=0
  if len(affectedCity)==0:
    try:
      numberofEmp=countByCountry.loc[[affectedCountry]]['Head Count'][affectedCountry]
    except:
      numberofEmp=1000
  else:
    try:
      numberofEmp=countByCity.loc[[affectedCity]]['Head Count'][affectedCity]
    except:
      numberofEmp=1000
  
  #Extracting Current Travellers count from Supporting Data
  travellers=pd.read_excel("/content/drive/MyDrive/EY_Hack_GDS_2.0/Travellers.xlsx")
  travellers['year']=pd.DatetimeIndex(travellers['Travel Date']).year
  travellers['month']=pd.DatetimeIndex(travellers['Travel Date']).month
  travellers=travellers.groupby(['year','month','Travel Location']).sum()
  currentTravellers=0
  try:
    currentTravellers=travellers.loc[(2020,1,affectedCountry)]['No. of travellers']
  except:
    currentTravellers=3 #this is the mean avg travellers of each country
  


  #Extracting baseline Score from Supporting Data
  baseline=pd.read_excel("/content/drive/MyDrive/EY_Hack_GDS_2.0/Baseline.xlsx")
  baseline.set_index('LOCATION',inplace=True)
  baselineScore=baseline.loc[[affectedCountry]][finalCat][affectedCountry]
  
  #predicting factors for calculation of Risk Score 
  riskscore=pd.read_excel(riskscore)
  riskscore.set_index('Geo Country',inplace=True)
  riskfactors={}
  riskfactors["Number_of_office"]=riskscore.loc[[affectedCountry]]['Number_of_office'][affectedCountry]
  riskfactors["Head Count"]=riskscore.loc[[affectedCountry]]['Head Count'][affectedCountry]
  riskfactors["Number_of_incident"]=riskscore.loc[[affectedCountry]]['Number_of_incident'][affectedCountry]
  riskfactors["Client_Presence"]=riskscore.loc[[affectedCountry]]['Client_Presence'][affectedCountry]
  if (sevValue=="Extreme"):
    riskfactors["Severity"]=5
  elif (sevValue=="High"):
    riskfactors["Severity"]=4
  elif (sevValue=="Medium"):
    riskfactors["Severity"]=3
  elif (sevValue=="Low"):
    riskfactors["Severity"]=2
  else:
    riskfactors["Severity"]=1

  expectedtravellers=random.randint(0,30)

  # for predicting number of upcoming incidents
  inci_count=riskscore.loc[[affectedCountry]]['Incident_Count'][affectedCountry]
  inci_count = inci_count.strip('][').split(', ')
  inci_count = map(int, inci_count)
  inci_count = list(inci_count)
  upcomingCases=prediction(inci_count,inci_count[-4:])
  fact_cases=0
  #print(upcomingCases["Prediction:"])
  for ele in range(0, len(upcomingCases["Prediction:"])):
    fact_cases = fact_cases + upcomingCases["Prediction:"][ele]
  
  #impact/esclation score
  impact_score= int (((fact_cases/60)*5*0.4) + ((baselineScore/6)*5*0.15) + ((expectedtravellers/30)*5*0.15) + (riskfactors["Head Count"]*0.15) + (riskfactors["Number_of_office"]*0.15))

  finalOutput={'Source:':source,'Incident Date:':incidentDate,'Title:':subject,'Description':incidentDescription,'Category':finalCat
             ,'Incident Country':affectedCountry,'Incident COuntries':affectedCountries,'Incident City':affectedCity,'Incident Cities':affectedCities
             ,'Severity':sevValue,'RiskCategory Prob':category,'EY Office Impacted':officeImpacted,'Number of Employees':numberofEmp,
             'Current Travellers':currentTravellers,'Baseline Score':baselineScore,"Expected Travellers":expectedtravellers,"RiskScore":riskfactors,"Impact Score":impact_score}
  print(finalOutput)

In [ ]:
#Calling the function
category('/content/drive/MyDrive/EY_Hack_GDS_2.0/abhi.pdf', '/content/drive/MyDrive/EY_Hack_GDS_2.0/Locations and Headcount.xlsx','/content/drive/MyDrive/EY_Hack_GDS_2.0/Severity.xlsx', '/content/drive/MyDrive/EY_Hack_GDS_2.0/Travellers.xlsx', '/content/drive/MyDrive/EY_Hack_GDS_2.0/Baseline.xlsx','/content/drive/MyDrive/EY_Hack_GDS_2.0/RiskScore.xlsx', text)


{'Source:': 'VCC', 'Incident Date:': '', 'Title:': 'travellersnow ', 'Description': ' datelistofaffectedindividualsandassetsintheimpactedarea exposureinimpactedareaonlylocationtherenowthereinnext24hoursthereinnext1 7daystherewhilethealertisactiveaffectedcities0000exposurebycountrylocationtherenowthereinnext24hoursthereinnext1 com sent saturday datetravelsecurityanalysisandmedicalinformationinthepalmofyourhands thisemailcontainsconfidentialinformationintendedfortheaddressee namedabove colombia barranquilla avoidvicinityofongoingprotestsduetounrestoverviewlastupdated avoidvicinityofongoingprotestsduetounrestoverviewlastupdated 01may202117 24 didyouknow downloadtheassistanceappforup datetravelsecurityanalysisandmedicalinformationinthepalmofyourhands downloadtheassistanceappforup datetravelsecurityanalysisandmedicalinformationinthepalmofyourhands thisemailcontainsconfidentialinformationintendedfortheaddressee saturday may1 20215 barranquilla avoidvicinityofongoingprotestsduetounrestovervi

In [ ]:
#Calling the function
category('/content/drive/MyDrive/EY_Hack_GDS_2.0/FilesWorked/Sample.txt', '/content/drive/MyDrive/EY_Hack_GDS_2.0/Locations and Headcount.xlsx','/content/drive/MyDrive/EY_Hack_GDS_2.0/Severity.xlsx', '/content/drive/MyDrive/EY_Hack_GDS_2.0/Travellers.xlsx', '/content/drive/MyDrive/EY_Hack_GDS_2.0/Baseline.xlsx','/content/drive/MyDrive/EY_Hack_GDS_2.0/RiskScore.xlsx', "z")

{'Source:': 'VCC', 'Incident Date:': '', 'Title:': 'travellersnow ', 'Description': ' colombia buenaventura anticipatedisruptionatseaportduetoongoingprotestbystrikingportworkersoverviewlastupdated datelistofaffectedindividualsandassetsintheimpactedarea exposureinimpactedareaonlylocationtherenowthereinnext24hoursthereinnext1 7daystherewhilethealertisactiveaffectedcities0000exposurebycountrylocationtherenowthereinnext24hoursthereinnext1 all24 hourassistancecentrephonenumberscanbeviewedhere yourfeedbackwasthisalertuseful downloadtheassistanceappforup datetravelsecurityanalysisandmedicalinformationinthepalmofyourhands thisemailcontainsconfidentialinformationintendedfortheaddressee buenaventura anticipatedisruptionatseaportduetoongoingprotestbystrikingportworkersoverviewlastupdated 30apr202122 pleaselogintotrackerforthelatestinformationorclickhereforanup datelistofaffectedindividualsandassetsintheimpactedarea exposureinimpactedareaonlylocationtherenowthereinnext24hoursthereinnext1 friday ap

In [ ]:
import os

In [ ]:
#Listing number of files to run
arr = os.listdir('/content/drive/MyDrive/EY_Hack_GDS_2.0/FilesWorked')

In [ ]:
#Predicting for n number of files at once
sucessful=[]
unsucessful=[]
for i in arr:
  filename='/content/drive/MyDrive/EY_Hack_GDS_2.0/FilesWorked'+'/'+i
  try:
    category(filename, '/content/drive/MyDrive/EY_Hack_GDS_2.0/Locations and Headcount.xlsx','/content/drive/MyDrive/EY_Hack_GDS_2.0/Severity.xlsx', '/content/drive/MyDrive/EY_Hack_GDS_2.0/Travellers.xlsx', '/content/drive/MyDrive/EY_Hack_GDS_2.0/Baseline.xlsx','/content/drive/MyDrive/EY_Hack_GDS_2.0/RiskScore.xlsx', "z")
    print(filename)
    sucessful.append(filename)
  except:
    unsucessful.append(filename)

{'Source:': 'ISOS', 'Incident Date:': '03 February 2021 21:15', 'Title:': ' travellers now - Advisory: Myanmar - Continue to monitor developments,', 'Description': ' november 2020 elections securityforces clashes pro flights resumefrom4 february yangonregions potential protest politicalinfluence sense latest speed internetservices restored repatriation flights resumefrom4 conflictcopyright 2020aea international reimpose economicsanctions myanmar fraudduring november 2020', 'Category': 'Civil', 'Incident Country': 'Myanmar', 'Incident COuntries': {'Myanmar': 8}, 'Incident City': 'Cali', 'Incident Cities': {'Cali': 2, 'Yangon': 1}, 'Severity': 'High', 'RiskCategory Prob': {'Political': 30, 'Security': 0, 'Travel': 23, 'Crime': 0, 'Civil': 38, 'Terrorism': 0, 'Medical': 0, 'Infrastructure': 7, 'Kidnap': 0, 'Institutional': 0}, 'EY Office Impacted': 1, 'Number of Employees': 280, 'Current Travellers': 3, 'Baseline Score': 5, 'Expected Travellers': 26, 'RiskScore': {'Number_of_office': 1, '

In [ ]:
sucessful

['/content/drive/MyDrive/EY_Hack_GDS_2.0/FilesWorked/ISOS Alert - 03 Feb.pdf',
 '/content/drive/MyDrive/EY_Hack_GDS_2.0/FilesWorked/CORE Incident Alert Myanmar - 01 Feb 2021(2).pdf',
 '/content/drive/MyDrive/EY_Hack_GDS_2.0/FilesWorked/ISOS Alert - 01 Feb.pdf',
 '/content/drive/MyDrive/EY_Hack_GDS_2.0/FilesWorked/CORE Alert - 18 Dec.pdf']